In [ ]:
import os

try:
    has_changed_dir
except:
    has_changed_dir = False

try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
except:
    IN_COLAB = False

if IN_COLAB:
    %pip install datasets
    %pip install translate-toolkit
    #%pip install bitsandbytes

    !git clone https://github.com/MarkusSibbesen/KDS-MI-temp.git

    if not has_changed_dir:
        os.chdir('KDS-MI-temp')
        has_changed_dir = True
else:
    if not has_changed_dir:
        os.chdir('.')
        has_changed_dir = True

In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
model_url = 'EleutherAI/pythia-14m'
device = 'cpu'

model_url = 'AI-Sweden-Models/gpt-sw3-356m'
device = 'cuda'

In [5]:
from make_steering_vectors import main

In [ ]:
main(
    model_url=model_url,
    target_language='da',
    complement_languages=['en', 'sv', 'is', 'nb'],
    device=device,
    out_folder='steering_vectors'
)

 32%|███▏      | 6/19 [00:12<00:26,  2.03s/it]


In [32]:
from utils.hooking import HookManager
from utils.compatibility import *
from utils.data import load_steering_vector, load_bible_data

from transformers import AutoModelForCausalLM, AutoTokenizer
from steering_loss import plot_loss_for_steering_vectors_batches, loss_with_steering_batches

from torch.utils.data import DataLoader

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_url).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_url)

In [24]:
ds = load_bible_data('en', 'da')

loader = DataLoader(ds, batch_size=32, shuffle=True)

In [28]:
hook_address = HookAddress.attention_pre.layer(4)
steering_vector = load_steering_vector('da', hook_address, model)

In [35]:
plot_loss_for_steering_vectors_batches(
    model,
    tokenizer,
    loader,
    5,
    'en',
    'da',
    50
)

1it [02:40, 160.55s/it]


KeyboardInterrupt: 